In [6]:
using Pkg
Pkg.activate("..") 
Pkg.instantiate()
using JPEC, Plots
gr() 



  Activating project at `~/git/JPEC5/JPEC`


Plots.GRBackend()

In [13]:
include("../src/DCON/fourfit.jl")
import .fourfit_make_metric, .MetricData, .fourfit_make_matrix, .MatrixData, .compute_eigenvalues

┌ Warning: Replacing docs for `Main.FourfitMetric.compute_eigenvalues :: Tuple{Main.FourfitMetric.MatrixData, Float64}` in module `Main.FourfitMetric`
└ @ Base.Docs docs/Docs.jl:243


In [19]:
# 1. Define the input parameters for the equilibrium solver.
#    - eq_filename: The name of the g-file we just created.
#    - eq_type: "efit" for a standard g-file.
#    - jac_type: "boozer" or "hamada" for the output coordinates.
#    - mpsi, mtheta: Resolution of the output grid.
equil_input = JPEC.Equilibrium.EquilInput(
    "beta_1.00",        # eq_filename
    "efit",          # eq_type
    "boozer",        # jac_type
    0.01,             # psilow
    
    0.99,             # psihigh
    100,             # mpsi (number of radial grid points)
    128              # mtheta (number of poloidal grid points)
)

# 2. Run the main equilibrium setup function.
#    This will read the file, solve the direct problem, and return the final object.
println("Starting equilibrium reconstruction...")
plasma_eq = JPEC.Equilibrium.setup_equilibrium(equil_input)
println("Equilibrium reconstruction complete.")

Starting equilibrium reconstruction...
--- Julia Equilibrium Setup ---
Equilibrium file: beta_1.00
Type = efit, Jac_type = boozer
----------------------------------------
--> Processing EFIT g-file: beta_1.00
--> Parsed from header: nw=129, nh=128
--> All main data blocks parsed successfully.
--> Creating 1D profile splines...
--> 1D Spline fitting complete.
--> Creating 2D psi spline...
--> 2D Spline fitting complete.
--- Starting Direct Equilibrium Processing ---
Finding magnetic axis...
  Iter  1: R = 3.115238, Z = 0.000008, |ΔR|=1.15e-01, |ΔZ|=1.11e-05
  Iter  2: R = 3.117635, Z = -0.000003, |ΔR|=2.40e-03, |ΔZ|=1.10e-05
  Iter  3: R = 3.117636, Z = -0.000003, |ΔR|=1.07e-06, |ΔZ|=1.04e-09
  Iter  4: R = 3.117636, Z = -0.000003, |ΔR|=2.12e-13, |ΔZ|=5.62e-16
Magnetic axis found at R=3.1176357129137466, Z=-2.9184696943092166e-6.
JPEC.SplinesMod.Helper.ReadOnlyArray{Float64, 1, Vector{Float64}}JPEC.SplinesMod.Helper.ReadOnlyArray{Float64, 1, Vector{Float64}}Array{Float64, 3}Finding inbo

In [20]:
# Fourfit Metric Calculation Example
println("📚 Fourfit Metric Calculation")
println("="^30)

using .FourfitMetric

try
    # Metric tensor calculation
    metric_result = FourfitMetric.fourfit_make_metric(
        plasma_eq.rzphi,  # 2D geometry spline
        plasma_eq.sq;     # 1D profile spline
        mpsi=100,          # Radial grid points
        mtheta=128,        # Poloidal grid points  
        mband=8,          # Fourier bandwidth
        fft_flag=true,    # Use FFT
        verbose=true      # Detailed output
    )
    
    println("✅ Metric calculation complete!")
    println("   Grid size: $(size(metric_result.fs))")
    
    # Evaluate at a specific point (if spline is available)
    if metric_result.fspline !== nothing
        psi_test = 0.5
        theta_test = π/2
        
        components = FourfitMetric.fspline_eval_metric(metric_result, psi_test, theta_test)
        println("   (ψ=$psi_test, θ=$(round(theta_test/π, digits=2))π)에서:")
        println("   g11 = $(round(components[1], digits=6))")
        println("   g22 = $(round(components[2], digits=6))")
        println("   jac = $(round(components[7], digits=6))")
    end
    
catch e
    println("❌Error: $e")
end

📚 Fourfit Metric Calculation
🔧 Starting metric tensor calculation...
   Grid: 101 × 129, mband: 8
📊 Computing metric tensor components on grid...
✅ Grid computation complete.
🔧 Fitting Fourier-spline representation...
✅ Fourier-spline fitting successful.

📈 Metric Tensor Summary:
   g11: [0.012399, 15.154528]
   g22: [0.028138, 38.132717]
   g33: [0.0, 5.179531]
   g23: [-11.223782, 1.223132]
   g31: [-5.376454, 5.376299]
   g12: [-16.747402, 16.745226]
   jmat: [25.286172, 60.274063]
   jmat1: [2.894421, 300.957259]
🎉 Metric tensor calculation complete!

✅ Metric calculation complete!
   Grid size: (101, 129, 8)
   (ψ=0.5, θ=0.5π)에서:
   g11 = 0.177241
   g22 = 1.387217
   jac = 29.683082


In [21]:
# Fourfit Matrix Calculation Example
println("🚀 Fourfit Matrix Calculation")
println("="^30)

using .FourfitMetric
using Interpolations


try
    # Step 1: Metric tensor calculation
    println("1️⃣  Calculating Metric Tensor...")
    metric_result = FourfitMetric.fourfit_make_metric(
        plasma_eq.rzphi,
        plasma_eq.sq;
        mpsi=100,
        mtheta=128,
        mband=2,
        fft_flag=false,
        verbose=true
    )
    
    # Step 2: Matrix calculation
    println("2️⃣  Calculating MHD Coefficient Matrix...")
    
    # Calculate psio (Toroidal flux normalization)
    psio = 2π  # Default value, should be obtained from equilibrium data
    
    matrix_result = FourfitMetric.fourfit_make_matrix(
        metric_result,
        plasma_eq.sq,
        plasma_eq.rzphi,
        psio;
        nn=1,           # Toroidal mode number
        mlow=-50,        # Minimum poloidal mode
        mhigh=50,        # Maximum poloidal mode
        power_flag=true,
        feval_flag=true,
        sas_flag=true
    )
    
    # Step 3: Analyze results
    println("3️⃣  Analyzing Results:")
    println("   Matrix dimensions: $(size(matrix_result.amat))")
    println("   Mode range: $(matrix_result.mlow) to $(matrix_result.mhigh)")
    println("   Number of modes: $(matrix_result.mpert)")
    
    # Example of eigenvalue computation
    if matrix_result.amat !== nothing
        eigenvals = FourfitMetric.compute_eigenvalues(matrix_result, 0.5)
        if length(eigenvals) > 0
            println("   First 3 eigenvalues:")
            for (i, λ) in enumerate(eigenvals[1:min(3, length(eigenvals))])
                println("     λ[$i] = $(round(real(λ), digits=6)) + $(round(imag(λ), digits=6))i")
            end
        end
    end
    
    println("✅ All calculations completed successfully!")
    
catch e
    println("❌ Error occurred: $e")
    println("\n🔍 Error Analysis:")
    if isa(e, UndefVarError)
        println("   Undefined variable error: $(e.var)")
        println("   This might be a module scope issue.")
    elseif isa(e, MethodError)
        println("   Method error: $(e.f)")
        println("   Argument types: $(typeof.(e.args))")
        println("   This might be an issue with spline evaluation.")
    elseif isa(e, BoundsError)
        println("   Array index out of bounds error")
        println("   Please check the array dimensions.")
    end
    println("\n💡 Suggestions:")
    println("   1. Ensure equilibrium data is loaded correctly.")
    println("   2. Check the return format of spline_eval.")
    println("   3. Verify all variables are defined in the correct scope.")
end


🚀 Fourfit Matrix Calculation
1️⃣  Calculating Metric Tensor...
🔧 Starting metric tensor calculation...
   Grid: 101 × 129, mband: 2
📊 Computing metric tensor components on grid...
✅ Grid computation complete.
🔧 Fitting Fourier-spline representation...
✅ Fourier-spline fitting successful.

📈 Metric Tensor Summary:
   g11: [0.012399, 15.154528]
   g22: [0.028138, 38.132717]
   g33: [0.0, 5.179531]
   g23: [-11.223782, 1.223132]
   g31: [-5.376454, 5.376299]
   g12: [-16.747402, 16.745226]
   jmat: [25.286172, 60.274063]
   jmat1: [2.894421, 300.957259]
🎉 Metric tensor calculation complete!

2️⃣  Calculating MHD Coefficient Matrix...
🔧 Starting MHD matrix calculation...
   Mode range: -50 to 50
   Toroidal mode: n = 1
   Bandwidth: 2
   ⚠️  Set mband to 0 for maximum stability
   Identity matrix imat initialized with imat[0]=1
📊 Computing coefficient matrices on flux surfaces...
   ✅ Extracted Fourier coefficients successfully
   Sample coefficients:
      g11[0] = 1.0 + 0.0im
      g22[0